%matplotlib inline

In [1]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import swifter
import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [2]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')

train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath
train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath

train_df = train_df.append(train_df_2)
del train_df_2
train_df = train_df[train_df['have_facemesh']]

In [3]:
class DataFrameBatchIterator:
    def __init__(self, dataframe, batch_size):
        self.df = dataframe
        self.index = 0
        self.batch_size = batch_size

    def __iter__(self):
        return self
        
    def __next__(self):
        if self.index < len(train_df):
            self.index += self.batch_size
            return self.df[self.index - self.batch_size: self.index].copy()
        else:
            raise StopIteration

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [5]:
INPUT_SIZE = (224, 224, 3)

In [6]:
mobilenet = tf.keras.applications.MobileNetV2(input_shape=INPUT_SIZE,
                                           include_top=False,
                                           weights='imagenet')

In [7]:
img_inputs = tf.keras.Input(shape=(INPUT_SIZE))
mobilenet_output = mobilenet(img_inputs)

In [8]:
x = tf.keras.layers.GlobalAveragePooling2D()(mobilenet_output)
for size in [640]:
    x = tf.keras.layers.Dense(size, activation = 'relu')(x)

In [9]:
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

In [10]:
model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name="mobilenet")

In [11]:
model.summary()

Model: "mobilenet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 640)               819840    
                                                                 
 dense_1 (Dense)             (None, 1)                 641       
                                                                 
Total params: 3,078,465
Trainable params: 3,044,353
Non-t

In [12]:
model.layers[1].trainable = False

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.MeanSquaredError()

In [14]:
batch_size = 128

In [15]:
import time
start_time = time.time()

total_loss = 0
total_step = 0
for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

    # Process Data
    imgs = row.subDirectory_filePath.swifter.progress_bar(False).apply(lambda x: cv2.resize(
        cv2.cvtColor(
            cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))

    img_array = np.array([])
    for img in list(imgs):
        img_array = np.append(img_array, img)

    img_array = img_array.reshape(-1, *list(imgs)[0].shape)

    y_arousal = np.array(row.arousal)

    # Train
    with tf.GradientTape() as tape:
        logits = model(img_array, training=True)
        loss_value = loss_fn(y_arousal, logits)
        
        total_loss += float(loss_value)
        total_step += 1

    grads = tape.gradient(loss_value, model.trainable_weights)

    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # Log every 200 batches.
    if step % 200 == 0:
        print(
            f"Training loss at step {step}: {total_loss/total_step}"
        )
        print("Seen so far: %s samples" % ((step + 1) * batch_size))
        print(f"--- {time.time() - start_time} seconds for iter {total_step} step, each set have {batch_size}, so the model train with {total_step*batch_size} img this iter ---")
        total_loss = 0
        total_step = 0
        start_time = time.time()

Training loss at step 0: 2.3288793563842773
Seen so far: 32 samples
--- 2.4843590259552 seconds for iter 1 step, each set have 32, so the model train with 32 img this iter ---
Training loss at step 200: 0.9393826459348201
Seen so far: 6432 samples
--- 61.62785744667053 seconds for iter 200 step, each set have 32, so the model train with 6400 img this iter ---


KeyboardInterrupt: 

In [ ]:
total_loss = 0

In [ ]:
total_loss



BATCH_SIZE=40 #512 #64 #32 #64

def save_model(model,filename):
    model.save_weights('weights.h5')
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)

    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights('weights.h5')
    loaded_model.save(filename)

base_model